# Basic usage

This notebook shows some basic usage of the genomic-features package.

In [1]:
import pandas as pd

import genomic_features as gf

## Retrieving Ensembl gene annotations

We can load annotation tables using {func}`genomic_features.ensembl.annotation`.

In [2]:
ensdb = gf.ensembl.annotation(species="Hsapiens", version="108")

These tables have been created for the [`ensembldb` Bioconductor package](https://bioconductor.org/packages/release/bioc/html/AnnotationHub.html) {cite:p}`Rainer_2019`, and are automatically downloaded and cached from the [`AnnotationHub`](https://bioconductor.org/packages/release/bioc/html/AnnotationHub.html) resource.

We can check which Ensembl versions are available for each species using the {func}`genomic_features.ensembl.list_versions` util.

In [5]:
gf.ensembl.list_versions(species='Mmusculus')

,Species,Ensembl_version
37,Mmusculus,v87
105,Mmusculus,v88
173,Mmusculus,v89
247,Mmusculus,v90
330,Mmusculus,v91
419,Mmusculus,v92
510,Mmusculus,v93
621,Mmusculus,v94
748,Mmusculus,v95
894,Mmusculus,v96


## Using annotations

In [3]:
genes = ensdb.genes()
genes.head()

,gene_id,gene_name,gene_biotype,gene_seq_start,gene_seq_end,seq_name,seq_strand,seq_coord_system,description,gene_id_version,canonical_transcript
0,ENSG00000000003,TSPAN6,protein_coding,100627108,100639991,X,-1,chromosome,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],ENSG00000000003.15,ENST00000373020
1,ENSG00000000005,TNMD,protein_coding,100584936,100599885,X,1,chromosome,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],ENSG00000000005.6,ENST00000373031
2,ENSG00000000419,DPM1,protein_coding,50934867,50959140,20,-1,chromosome,dolichyl-phosphate mannosyltransferase subunit...,ENSG00000000419.14,ENST00000371588
3,ENSG00000000457,SCYL3,protein_coding,169849631,169894267,1,-1,chromosome,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,ENSG00000000457.14,ENST00000367771
4,ENSG00000000460,C1orf112,protein_coding,169662007,169854080,1,1,chromosome,chromosome 1 open reading frame 112 [Source:HG...,ENSG00000000460.17,ENST00000359326


In [4]:
ensdb.chromosomes()

,seq_name,seq_length,is_circular
0,X,156040895,0
1,20,64444167,0
2,1,248956422,0
3,6,170805979,0
4,3,198295559,0
...,...,...,...
452,LRG_741,231167,0
453,LRG_763,176286,0
454,LRG_792,42144,0
455,LRG_793,38439,0


### Adding annotations to an AnnData object:

In [5]:
import scanpy as sc
pbmc = sc.datasets.pbmc3k()

In [6]:
pbmc.var.head()

,gene_ids
index,
MIR1302-10,ENSG00000243485
FAM138A,ENSG00000237613
OR4F5,ENSG00000186092
RP11-34P13.7,ENSG00000238009
RP11-34P13.8,ENSG00000239945


In [7]:
pbmc.var = (
    pd.merge(
        (
            pbmc.var
            .reset_index()
            .rename(columns={"gene_ids": "gene_id", "index": "orig_gene_name"})
        ),
        genes,
        on="gene_id",
        how="left",
    )
    .set_index("gene_id")
)

In [8]:
pbmc.var.head()

,orig_gene_name,gene_name,gene_biotype,gene_seq_start,gene_seq_end,seq_name,seq_strand,seq_coord_system,description,gene_id_version,canonical_transcript
gene_id,,,,,,,,,,,
ENSG00000243485,MIR1302-10,MIR1302-2HG,lncRNA,29554.0,31109.0,1,1.0,chromosome,MIR1302-2 host gene [Source:HGNC Symbol;Acc:HG...,ENSG00000243485.5,ENST00000473358
ENSG00000237613,FAM138A,FAM138A,lncRNA,34554.0,36081.0,1,-1.0,chromosome,family with sequence similarity 138 member A [...,ENSG00000237613.2,ENST00000417324
ENSG00000186092,OR4F5,OR4F5,protein_coding,65419.0,71585.0,1,1.0,chromosome,olfactory receptor family 4 subfamily F member...,ENSG00000186092.7,ENST00000641515
ENSG00000238009,RP11-34P13.7,,lncRNA,89295.0,133723.0,1,-1.0,chromosome,novel transcript,ENSG00000238009.6,ENST00000477740
ENSG00000239945,RP11-34P13.8,,lncRNA,89551.0,91105.0,1,-1.0,chromosome,novel transcript,ENSG00000239945.1,ENST00000495576
